**поиск в масиве текста с помощью W2V**

Евгений Борисов <esborisov@sevsu.ru>

---

https://www.youtube.com/watch?v=XlEHTf93Y8w

https://www.youtube.com/watch?v=aZ5se_SW81c

https://github.com/girafe-ai/ml-mipt/blob/master/week1_01_word_embeddings/week01_fun_with_embeddings.ipynb

---


__задача:__ реализовать поиск строк по запросам

__решение:__
- кодируем слова w2v (Gensim)
- для каждой строки агрегируем список кодов (считаем устреднённый вектор w2v)
- кодируем слова запроса и агрегируем список кодов
- ищем ближайшие векторы вопросов к вектору запроса (косинусная мера близости)

## загружаем текст

In [1]:
import gzip
import re
import numpy as np
from numpy import random as rng

In [2]:
# загружаем текст
with gzip.open('../data/dostoevsky-besy-p2.txt.gz','rt',encoding='utf-8') as f: text = f.read()
# print(text[100:1000])
text = re.split(r'\n+', re.sub(r'([.;?!])',r'\1\n',text) ) 
print('\n\nСТРОК:',len(text))



СТРОК: 7925


In [3]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е',] # замена ё для унификации
       ,[r'[^а-я ]+',' '] # оставляем только буквы и пробел
       ,[r'^- *',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

text = list(map(string_normalizer,text))

# разрезаем текст на слова
text = [ [ w.strip() for w in s.split() if w.strip() ] for s in text ]

In [4]:
# удаляем короткие и очень длинные строки
print( len(text) )
text = [ s for s in text if 7<len(s)<17  ]
print( len(text) )

7925
2238


In [5]:
for i in rng.permutation(len(text))[:10]:   print(' '.join( text[i]) )

я знаю что даже кириллов который к ним почти вовсе не принадлежит доставил об вас сведения
другого впрочем оттенка не будет вовсе приняты меры
одно из самых последних изящнейших беллетристических вдохновений семена егоровича оно называется
он топнул опять ногой слюня брызгала с его губ
я во первых вовсе не такой уж мягкий а во вторых
вскричал я догадавшись старые басни да неужто вы верили до сих пор
я вас предупредил чтобы вы все таки имели в виду
на некоторые жалобы и запросы положено было систематически не отвечать
а у вас так тщательно сохранилось это письмо
с вашим умом вы бы могли понять это в негодовании пробормотал шатов


## загружаем W2V

https://nlpub.ru/Russian_Distributional_Thesaurus

In [6]:
#!wget -c http://panchenko.me/data/dsl-backup/w2v-ru/tenth.norm-sz500-w7-cb0-it5-min5.w2v

In [7]:
from gensim.models import KeyedVectors 

In [8]:
%%time 

w2v_file = 'w2v/tenth.norm-sz500-w7-cb0-it5-min5.w2v'
w2v_model = KeyedVectors.load_word2vec_format(w2v_file, binary=True, unicode_errors='ignore')
w2v_model.init_sims(replace=True)

CPU times: user 13.1 s, sys: 2.43 s, total: 15.5 s
Wall time: 15.5 s


In [9]:
words = list(w2v_model.vocab.keys())
vocab_size = len(words)
print('Vocab size', vocab_size)

Vocab size 2641862


In [10]:
for i in rng.permutation(vocab_size)[:10]:
    w = words[i]
    ww = [ v[0] for v in w2v_model.most_similar(w,topn=5) ]
    print( w,':',ww )

устроимъ : ['медоваромъ', 'поeхать', 'увѣряю', 'увeрены', 'лeснымъ']
тяжек : ['тяжел', 'тяжкий', 'долог', 'труден', 'томителен']
запази : ['означаваше', 'ласанин', 'стайнър', 'тъга', 'задължения']
нащупаете : ['—\xa0просуньте', 'ощупывайте', 'ощупайте', 'уплотните', 'простучите']
regionis : ['gotthi', 'halani', 'transeuntes', 'athanaricus', 'amplae']
танька, — : ['полинка,\xa0—', 'анька,\xa0—', 'ираида,\xa0—', '—\xa0командирша', 'иришка,\xa0—']
ралайята : ['зиккур', 'дармштайн', 'гонзорской', 'элтайна', 'абадосса']
неманципируемые : ['манципируемые', 'mancipi', 'оборотоспособности', 'суперфиций', 'непотребляемые']
коралин : ['тайах,\xa0–', 'поторопить,\xa0–', 'упрямица,\xa0–', 'риба,\xa0–', '–\xa0извини,–']
посвящен, – : ['автарх,\xa0–', 'недостаточно…\xa0–', 'брек,\xa0–', 'кейптауне,\xa0–', 'куинс,\xa0–']


## кодируем текст

In [11]:
# кодируем слова w2v (Gensim)

In [17]:
def w2v_words_encode(t): 
    return np.array([ w2v_model.get_vector(w) 
                     for w in t 
                     if w in w2v_model.vocab.keys() ])

In [18]:
i = rng.randint(len(text))

print( ' '.join(text[i]), ' : ' , len(text[i]), 'слов' )

x = w2v_words_encode( text[i] )
x.shape

петр степанович даже не посмотрел на нее взял ножницы и начал возиться с ними  :  14 слов


(14, 500)

----

In [14]:
# для каждой строки агрегируем список кодов 
# (считаем устреднённый вектор w2v, как вариант - w2v*idf)

In [15]:
# кодируем слова запроса и агрегируем список кодов

In [16]:
# ищем ближайшие векторы вопросов к вектору запроса (косинусная мера близости)